<a href="https://colab.research.google.com/github/LatiefDataVisionary/multivariate-analysis-college-task/blob/main/src/DiscriminantAnalysis_week12_manually.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Library Importation and Data Loading**

Pada bagian awal ini, dilakukan impor terhadap pustaka-pustaka Python yang dibutuhkan dan pemuatan dataset.

1.  **Impor Pustaka**:
    *   **Pandas**: Digunakan untuk manipulasi data tabular, khususnya untuk membaca file CSV dan bekerja dengan struktur DataFrame.
    *   **NumPy**: Merupakan pustaka fundamental untuk komputasi numerik, menyediakan dukungan untuk operasi array dan matriks yang efisien, yang akan sangat berguna dalam perhitungan manual LDA.
2.  **Pemuatan Dataset**:
    *   Dataset dimuat dari URL yang disediakan (`https://raw.githubusercontent.com/LatiefDataVisionary/multivariate-analysis-college-task/refs/heads/main/datasets/dataset_LDA_2clusters.csv`) menggunakan fungsi `pd.read_csv()` dari Pandas.
    *   Hasil pemuatan data disimpan dalam sebuah DataFrame Pandas yang dinamakan `df`.
3.  **Inspeksi Awal Data**:
    *   `df.head()`: Menampilkan lima baris pertama dari DataFrame untuk mendapatkan gambaran awal mengenai struktur dan isi data.
    *   `df.info()`: Memberikan ringkasan informasi tentang DataFrame, termasuk jumlah entri, tipe data setiap kolom, dan jumlah nilai non-null. Ini berguna untuk memeriksa apakah ada data yang hilang atau tipe data yang tidak sesuai.
    *   `df.shape`: Menampilkan dimensi dari DataFrame (jumlah baris, jumlah kolom).
    *   `df.describe()`: Menampilkan statistik deskriptif dasar untuk kolom-kolom numerik dalam DataFrame, seperti mean, standar deviasi, nilai minimum, maksimum, dan kuartil.

Langkah-langkah ini penting untuk memastikan bahwa data telah dimuat dengan benar dan untuk mendapatkan pemahaman awal tentang dataset sebelum melanjutkan ke tahap analisis dan perhitungan selanjutnya.

### **1.1. Import library yang diperlukan**

In [1]:
import pandas as pd
import numpy as np